### 국회의원 정보 스크래핑
* 이름, ID 추출
* ID를 사용해서 상세페이지로 요청 보내기
* DataFrame에 저장하고, DB에 저장
* 시각화

In [9]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do'
req_param = {
    #?currentPage=1&rowPerPage=500
    "currentPage":1,
    "rowPerPage":500
}

res = requests.get(url, params=req_param)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    atag_list = soup.select("div.memberna_list dl dt a")
    
    member_list = []    
    for atag in atag_list:
        member_dict = {}
        
        member_dict['name'] = atag.text
        
        href = atag['href']
        matched = re.search(r'(\d+)', href)
        member_dict['id'] = matched.group(0)
        
        #print(member_dict)
        member_list.append(member_dict)

print(len(member_list))       

200
297
